<a href="https://colab.research.google.com/github/Arnav382/sentiment-analysis/blob/main/lstm_sentiment_analysis_keras_(news_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download("stopwords")
nltk.download('punkt')
stop_words= set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


function to remove stopwords

In [4]:
def remove_stopwods(df,label):
  j=0
  for i in df[label]:
    text_tokens=word_tokenize(i)
    filtered_words=[word for word in text_tokens if word not in stop_words]
    df[label][j]=filtered_words
    j=j+1

Importing DataSet of news

In [5]:
  # #Arnav Directory
  # from google.colab import drive 
  # drive.mount("/content/gdrive")
  # data = pd.read_csv("/content/gdrive/My Drive/JioMeetSentiments-2.csv")
  # test_data=pd.read_csv("/content/gdrive/My Drive/NewsData.csv")

In [6]:
 # rajat directory
from google.colab import drive 
drive.mount("/content/gdrive")
data = pd.read_csv("/content/gdrive/My Drive/jio_data/jio_news_finaldata.csv")

Mounted at /content/gdrive


In [7]:
data

,title,sentiment_true_label_str,sentiment_true_label,parsed_content
0,Air India passenger data breach reveals SITA h...,neutral,1.0,Three months after air transport data giant S...
1,India's Bharti Airtel swings to quarterly prof...,neutral,1.0,A man wearing a protective face shield cleans...
2,India private equity investments hit record hi...,positive,2.0,A broker reacts while trading at his computer...
3,Analysis: Frequent run-ins with India gov't cl...,neutral,1.0,3D printed Facebook and Twitter logos are pla...
4,[Update: June 05] Apple iOS 14/iPadOS 14 updat...,neutral,1.0,New updates are being added at the bottom of ...
...,...,...,...,...
759,Bollywood actress files plea against 5G implem...,neutral,1.0,"Juhi Chawla gripped by, well, Darr! One of the..."
760,"What’s up with… Amazon & MGM, French media gia...",neutral,1.0,"May 18, 2021 The speculation linking Amazon t..."
761,"Reliance AGM next week: From 5G launch, Jio Bo...",neutral,1.0,Jump to navigation Speak Now We are just a w...
762,"Airtel 5G network trial goes live in Gurgaon, ...",neutral,1.0,The trial delivered a peak speed of over 1Gbp...


In [8]:
data=data[["title","parsed_content","sentiment_true_label_str","sentiment_true_label"]]

In [9]:
data['parsed_content']= data['parsed_content'].astype("str")
data['title']= data['title'].astype("str")


In [10]:
data['sentiment_true_label']=data['sentiment_true_label'].fillna(1)
data['sentiment_true_label']=data['sentiment_true_label'].astype("int")

In [11]:
data.head()

,title,parsed_content,sentiment_true_label_str,sentiment_true_label
0,Air India passenger data breach reveals SITA h...,Three months after air transport data giant S...,neutral,1
1,India's Bharti Airtel swings to quarterly prof...,A man wearing a protective face shield cleans...,neutral,1
2,India private equity investments hit record hi...,A broker reacts while trading at his computer...,positive,2
3,Analysis: Frequent run-ins with India gov't cl...,3D printed Facebook and Twitter logos are pla...,neutral,1
4,[Update: June 05] Apple iOS 14/iPadOS 14 updat...,New updates are being added at the bottom of ...,neutral,1


In [12]:
remove_stopwods(data,"title")
remove_stopwods(data,"parsed_content")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
data.head()

,title,parsed_content,sentiment_true_label_str,sentiment_true_label
0,"[Air, India, passenger, data, breach, reveals,...","[Three, months, air, transport, data, giant, S...",neutral,1
1,"[India, 's, Bharti, Airtel, swings, quarterly,...","[A, man, wearing, protective, face, shield, cl...",neutral,1
2,"[India, private, equity, investments, hit, rec...","[A, broker, reacts, trading, computer, termina...",positive,2
3,"[Analysis, :, Frequent, run-ins, India, gov't,...","[3D, printed, Facebook, Twitter, logos, placed...",neutral,1
4,"[[, Update, :, June, 05, ], Apple, iOS, 14/iPa...","[New, updates, added, bottom, story…, Original...",neutral,1


In [14]:
def convertList(list1):  
    str = ''  # initializing the empty string  
    for i in list1: #Iterating and adding the list element to the str variable  
      str =str+ " "+i
    return str

In [15]:
k=0
while(k<len(data['title'])):
  data['title'][k]= convertList(data['title'][k])
  data['parsed_content'][k]= convertList(data['parsed_content'][k])
  k=k+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
data['title'][0]

' Air India passenger data breach reveals SITA hack worse first thought'

In [17]:
data["parsed_content"][0]

' Three months air transport data giant SITA reported data breach , still learning damage . Air India said week personal data 4.5 million passengers compromised following incident SITA , Indian flag carrier airline ’ data processor . The stolen information included passengers ’ names , credit card details , date birth , contact information , passport information , ticket information , Star Alliance Air India frequent flyer data , Air India said statement ( PDF ) . CVV/CVC data credit cards held SITA , said Air India urged passengers change passwords “ wherever applicable ensure safety personal data. ” The attack compromised data passengers registered Indian airline past decade , August 26 , 2011 February 3 , 2021 , Air India said statement . The revelation comes months SITA said suffered data breach involved passenger data . At time , SITA said notified several airlines — Malaysia Airlines , Finnair , Singapore Airlines , Jeju Air , Cathay Pacific , Air New Zealand , Lufthansa — breach

In [18]:
data['parsed_content'] = data['parsed_content'].apply(lambda x: x.lower())
data['parsed_content'] = data['parsed_content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


max_fatures = 25000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['parsed_content'].values)
X = tokenizer.texts_to_sequences(data['parsed_content'].values)
X = pad_sequences(X)

In [19]:
data=data.drop(["sentiment_true_label_str"],axis=1)
data

,title,parsed_content,sentiment_true_label
0,Air India passenger data breach reveals SITA ...,three months air transport data giant sita re...,1
1,India 's Bharti Airtel swings quarterly profi...,a man wearing protective face shield cleans b...,1
2,India private equity investments hit record h...,a broker reacts trading computer terminal sto...,2
3,Analysis : Frequent run-ins India gov't cloud...,3d printed facebook twitter logos placed comp...,1
4,[ Update : June 05 ] Apple iOS 14/iPadOS 14 u...,new updates added bottom story original story...,1
...,...,...,...
759,Bollywood actress files plea 5G implementatio...,juhi chawla gripped well darr one popular ...,1
760,"What ’ with… Amazon & MGM , French media gian...",may 18 2021 the speculation linking amazon m...,1
761,"Reliance AGM next week : From 5G launch , Jio...",jump navigation speak now we week away 44th r...,1
762,"Airtel 5G network trial goes live Gurgaon , s...",the trial delivered peak speed 1gbps accordi...,1


In [20]:
embed_dim = 128  
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3333, 128)         3200000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 3333, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 3,455,391
Trainable params: 3,455,391
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
Y = pd.get_dummies(data['sentiment_true_label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

In [22]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
10/10 - 270s - loss: 0.8553 - accuracy: 0.8036
Epoch 2/5
10/10 - 239s - loss: 0.4275 - accuracy: 0.8805
Epoch 3/5
10/10 - 244s - loss: 0.4190 - accuracy: 0.8805
Epoch 4/5
10/10 - 242s - loss: 0.4027 - accuracy: 0.8789
Epoch 5/5
10/10 - 242s - loss: 0.3671 - accuracy: 0.8805


Extracting a validation set, and measuring score and accuracy.

## Finding accuracy of the model using metrics


In [23]:
pred1=model.predict(X_test)
pred2=list()
for i in pred1:
 pred2.append(np.argmax(i))
pred2=np.array(pred2)
pred2  # prediction numpy array
# pred1_news=model.predict(x_test_headline)
# pred1_news

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### it gave all 1 (neutral sentiment) because our training data has 94% (approx ) neutral which made our model highly biased towards neutral. Also training dataset was also very small

Converting y_test to again its label for finding accuracy

In [24]:
Y_test1=[]
for i in Y_test:
  if np.argmax(i)==0:
    Y_test1.append(0)
  elif np.argmax(i)==1:
    Y_test1.append(1)
  else :
    Y_test1.append(2)
Y_test1=np.array(Y_test1)
Y_test1 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [25]:
from sklearn.metrics import f1_score
f1=f1_score(Y_test1,pred2,average='weighted')
f1

0.9126559714795008

## Prediction another data

In [63]:
data_test = pd.read_csv("/content/gdrive/My Drive/jio_data/JioMeetSentiments.csv")

In [64]:
data_test=data_test[["sentimenttype","simplifiedtext"]]
data_test

,sentimenttype,simplifiedtext
0,2,SUPER APP
1,1,PL ALLOW REGULAR MEETINGS WITH SAME LINK AND ...
2,0,PROVIDE WHITE BOARD
3,2,THIS IS AN BEST APP I LIKE IT
4,2,EASY TO USE AND VERY SMART APP
...,...,...
23722,2,MERA JIO MEET ON KU NHI HO RHA
23723,2,AWESOME JUST LOVE IT WAY BETTER THAN OTHER E...
23724,0,NO S� PUEDE CAMBIAR EL IDIOMA A ESPA�OL
23725,0,IT S TOO GOOD BUT I REQUEST YOU TO MAKE UPTIL...


In [65]:
data_test['simplifiedtext']= data_test['simplifiedtext'].astype("str")
data_test['sentimenttype']=data_test['sentimenttype'].fillna(1)
data_test['sentimenttype']=data_test['sentimenttype'].astype("int")

In [66]:
remove_stopwods(data_test,"simplifiedtext")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
k=0
while(k<len(data_test['simplifiedtext'])):
  data_test['simplifiedtext'][k]= convertList(data_test['simplifiedtext'][k])
  k=k+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
data_test['simplifiedtext'] = data_test['simplifiedtext'].apply(lambda x: x.lower())
data_test['simplifiedtext'] = data_test['simplifiedtext'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [69]:
max_fatures = 250000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data_test['simplifiedtext'].values)
X_testdata = tokenizer.texts_to_sequences(data_test['simplifiedtext'].values)
X_testdata = pad_sequences(X_testdata)

In [70]:
pred_test=model.predict(X_testdata)
pred_testf=list()
for i in pred_test:
 pred_testf.append(np.argmax(i))
pred_testf=np.array(pred_testf)
pred_testf

array([1, 1, 1, ..., 1, 1, 1])

In [71]:
np.unique(pred_testf)

array([1])

In [72]:
np.sort(pred_testf)

array([1, 1, 1, ..., 1, 1, 1])

In [73]:
np.count_nonzero(pred_testf)

23727